In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
train = pd.read_csv('/content/drive/My Drive/Hackathon/janatahack/files/train_SOaYf6m/train.csv')
test = pd.read_csv('/content/drive/My Drive/Hackathon/janatahack/files/test_vc2kHdQ.csv')

In [4]:
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('/content/drive/My Drive/Hackathon/janatahack/files/train_SOaYf6m/images/'+train['image_names'][i], 
                         target_size=(224,224,3), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|██████████| 1646/1646 [07:21<00:00,  3.73it/s]


In [0]:
y = train['emergency_or_not'].values
y = to_categorical(y)

In [6]:
len(y)

1646

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [0]:
train_datagen = ImageDataGenerator(shear_range=0.2, zoom_range=0.2, horizontal_flip=True, rotation_range=40)

In [0]:
val_datagen = ImageDataGenerator()

In [13]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(224,224,3),padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 110, 110, 64)      18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 55, 55, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 53, 53, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 24, 24, 128)      

In [0]:
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [0]:
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)

In [25]:
batch_size = 32
h = model.fit_generator(train_generator, epochs=30, steps_per_epoch=len(X_train) // batch_size, 
                        validation_data=(X_test, y_test),
                        callbacks=[keras.callbacks.ModelCheckpoint('/content/drive/My Drive/Hackathon/janatahack/files/modelbest_{val_accuracy:.4f}.h5',
                                                                   save_best_only=True,
                                                                   save_weights_only=False,
                                                                   monitor='val_accuracy')]
)

Epoch 1/30
41/41 [==============================] - 143s 3s/step - loss: 0.3867 - accuracy: 0.8259 - val_loss: 0.4343 - val_accuracy: 0.8045
Epoch 2/30
41/41 [==============================] - 143s 3s/step - loss: 0.3736 - accuracy: 0.8294 - val_loss: 0.5181 - val_accuracy: 0.8030
Epoch 3/30
41/41 [==============================] - 142s 3s/step - loss: 0.3769 - accuracy: 0.8279 - val_loss: 0.4101 - val_accuracy: 0.8258
Epoch 4/30
41/41 [==============================] - 145s 4s/step - loss: 0.3732 - accuracy: 0.8376 - val_loss: 0.6261 - val_accuracy: 0.7924
Epoch 5/30
41/41 [==============================] - 142s 3s/step - loss: 0.3650 - accuracy: 0.8357 - val_loss: 0.4699 - val_accuracy: 0.8061
Epoch 6/30
41/41 [==============================] - 142s 3s/step - loss: 0.3565 - accuracy: 0.8333 - val_loss: 0.4764 - val_accuracy: 0.8106
Epoch 7/30
41/41 [==============================] - 143s 3s/step - loss: 0.3413 - accuracy: 0.8470 - val_loss: 0.4177 - val_accuracy: 0.8106
Epoch 8/30
41

In [26]:
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('/content/drive/My Drive/Hackathon/janatahack/files/train_SOaYf6m/images/'+test['image_names'][i], 
                         target_size=(224,224,3), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

100%|██████████| 706/706 [03:20<00:00,  3.53it/s]


In [0]:
import tensorflow as tf
model = tf.keras.models.load_model('/content/drive/My Drive/Hackathon/janatahack/files/modelbest_0.8652.h5')

In [28]:
prediction = model.predict_classes(test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [0]:
test_again = pd.read_csv('/content/drive/My Drive/Hackathon/janatahack/files/test_vc2kHdQ.csv')

In [0]:
new_data = {'image_names':test_again.image_names.values, 'emergency_or_not':prediction}
submit = pd.DataFrame(new_data)

In [31]:
submit

,image_names,emergency_or_not
0,1960.jpg,0
1,668.jpg,1
2,2082.jpg,0
3,808.jpg,1
4,1907.jpg,0
...,...,...
701,674.jpg,1
702,1027.jpg,0
703,447.jpg,1
704,2176.jpg,0


In [0]:
submit.to_csv('submission3.csv', index=False)